In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import datetime
import pickle
import os.path

In [2]:
np.random.seed(1)

In [3]:
# shamelessly taken from https://danijar.com/variable-sequence-lengths-in-tensorflow/
def getSequenceLengths(batchOfSequences):
  used = tf.sign(tf.reduce_max(tf.abs(batchOfSequences), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

In [ ]:
import argparse

parser = argparse.ArgumentParser(description = "Please insert the train flag")
parser.add_argument('-t', '--train', action = "store",
                    help='If true, we train and save. Else, otherwise.', required = True)

my_args = vars(parser.parse_args())
trainFlag = my_args['train']
trainFlag = trainFlag.lower() in ("True", "t", "true", "1", 1)

In [4]:
logging = tf.logging

In [5]:
tf.logging.set_verbosity(tf.logging.INFO)

In [6]:
print(datetime.datetime.now())

2017-08-29 21:48:29.524345


In [7]:
df = np.load('invokerItems.npy')
match_ids = pd.read_pickle('final_processed_invoker_data')['matchId']

x = np.random.rand(df.shape[0])
mask = np.where(x < 0.75)[0]
mask2 = np.where(x >= 0.75)[0]
df_train = df[mask, :]
df_test = df[mask2, :]
match_ids_train = match_ids.iloc[mask]
match_ids_test = match_ids.iloc[mask2]

In [8]:
# 265 is highest ID, and we need a zero for empty
NUM_ITEMS = 266
DIM = NUM_ITEMS

# to make batch sizes the same for the hidden state of the LSTM cell when doing test vs train
# test is smaller than train
batchSize = int(len(df_test))
batched_ItemData = tf.placeholder(tf.float32, shape=[None, None, DIM])

batched_ItemData = tf.pad(batched_ItemData, [[0, 0], [0, 1], [0, 0]], "CONSTANT")
# shift each sequence to the left by one (stride to the right by 1)
# and then pad at the end to maintain same lengths!
batched_ItemData_strided = tf.strided_slice(batched_ItemData, [0, 1, 0], tf.shape(batched_ItemData))
targets = tf.pad(batched_ItemData_strided, [[0, 0], [0, 1], [0, 0]], "CONSTANT")
MAX_SequenceLength = df.shape[1]
batched_ItemDataSequenceLengths = getSequenceLengths(batched_ItemData)
lstmHiddenSize = 100
cell = tf.contrib.rnn.LSTMCell(lstmHiddenSize, cell_clip = 5)

In [9]:
state = tf.Variable(cell.zero_state(batchSize, tf.float32), trainable=False, name='lstm_state')

lstmOutput, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float32,
    sequence_length=batched_ItemDataSequenceLengths,
    inputs=batched_ItemData,
    initial_state = tf.nn.rnn_cell.LSTMStateTuple(state[0], state[1]))

# the assign is run every time we run on lstmOutput because of the with
with tf.control_dependencies([state.assign(last_states)]):
    lstmOutput = tf.identity(lstmOutput)

In [10]:
lstmOutput = tf.reshape(lstmOutput, [-1, lstmHiddenSize])

In [11]:
with tf.variable_scope('LayerToLogits'):
    weights = tf.Variable(tf.random_normal([lstmHiddenSize, NUM_ITEMS], stddev = 1.0/NUM_ITEMS/100), name='weightsLayer')
    bias = tf.Variable(tf.random_normal([NUM_ITEMS], stddev = 1.0/NUM_ITEMS/100), name='biasLayer')

In [12]:
logits = tf.matmul(lstmOutput, weights) + bias
targets = tf.reshape(targets, [-1, DIM])

with tf.variable_scope('accuracy'):
    with tf.variable_scope('correct_prediction'):
        correct_predictions = tf.equal(tf.argmax(logits, 1), tf.argmax(targets, 1))
        correct_predictions = tf.reshape(correct_predictions, shape=[batchSize, MAX_SequenceLength])
        correct_predictions = tf.cast(correct_predictions, tf.float32)
    with tf.variable_scope('accuracy'):
        accuracyPerSequence = tf.reduce_sum(correct_predictions, reduction_indices=1) / \
            tf.cast(batched_ItemDataSequenceLengths, tf.float32)
        averageAccuracy =  tf.reduce_mean(accuracyPerSequence)

loss = tf.nn.softmax_cross_entropy_with_logits(labels = targets, logits = logits)
mask = tf.sign(tf.reduce_max(tf.abs(targets), 1))
loss = mask * loss
loss = tf.reshape(loss, shape=[batchSize, MAX_SequenceLength])
# loss is 2d in a dynamic length way and then make it 1d to then reduce on
loss_per_sequence = tf.reduce_sum(loss, reduction_indices=1) / tf.cast(batched_ItemDataSequenceLengths, tf.float32)
cost = tf.reduce_mean(loss_per_sequence)

In [13]:
# default of 0.001
learning_rate = 0.001
# default of 0.99
beta1 = 0.99
# default of 0.999
beta2 = 0.999
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1, beta2 = beta2)

global_step = tf.Variable(0, name='global_step', trainable=False)

gradients, variables = zip(*optimizer.compute_gradients(cost))
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
train_op = optimizer.apply_gradients(zip(gradients, variables))

** Generate summary information **

In [14]:
# code from tensorflow documentation
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
    
with tf.name_scope('weights'):
    variable_summaries(weights)
    
with tf.name_scope('bias'):
    variable_summaries(bias)

with tf.name_scope("loss"):
    variable_summaries(loss_per_sequence)
    # this number is already calculated in variable_summaries of loss_per_sequence but be explicit
    tf.summary.scalar('cost', cost)
    
with tf.name_scope("accuracy"):
    variable_summaries(accuracyPerSequence)
    # this number is already calculated in variable_summaries of accuracyPerSequence but be explicit
    tf.summary.scalar('accuracy', averageAccuracy)

summary_op = tf.summary.merge_all()

In [15]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
ckpoint_dir = os.path.join(os.getcwd(), 'model-backups/model.ckpt')
print(ckpoint_dir)

/home/nabeel/Documents/Projects/Dota-ItemSequence/model-backups/model.ckpt


In [16]:
def train(sess):
    numEpochs = 1000
    numBatches = 4
    for epochIter in range(numEpochs):
        print('Epoch: {0}'.format(epochIter))
        for batchItr in range(numBatches):
            samp = np.random.choice(np.arange(df_train.shape[0]), size=batchSize, replace=False, p=None)
            if (epochIter+1) % 5 == 0:
                summaries, _ = sess.run([summary_op, train_op], 
                                         feed_dict = {batched_ItemData : df_train[samp, :, :]})
                train_writer.add_summary(summaries, epochIter)
                saver.save(sess, ckpoint_dir)
            else:
                sess.run(train_op, feed_dict = {batched_ItemData : df_train[samp, :, :]})

In [17]:
def test(sess):
    summaries, accuracies_val, cost_val = sess.run([summary_op, accuracyPerSequence, cost], 
                                         feed_dict = {batched_ItemData : df_test})
    #print('Accuracy: {0}'.format(averageAccuracy_val))
    print('Loss: {0}'.format(cost_val))
    test_writer.add_summary(summaries)
    return accuracies_val

In [56]:
def encodeItem(itemId):
    return np.array([int(i==int(itemId)) for i in range(NUM_ITEMS)])

filehandle = open("itemDict.obj",'rb')
itemDict = pickle.load(filehandle)

def lstmOutputToOneHotItemTensor(sess, lstmOutput):
    logit = sess.run(tf.matmul(lstmOutput, weights) + bias)
    return tf.cast(tf.reshape(encodeItem(np.argmax(logit)), [1, NUM_ITEMS]), tf.float32)

# takes the lstm output, takes it to logit, and returns the one hot
def decodeItem(sess, oneHotItem, itemDict):
    # oneHotItem is a tensor of rank 2, shape[1, NUM_ITEMS]
    val = sess.run(oneHotItem)
    return itemDict[np.asscalar(np.where(val[0] == 1)[0])]

def generate(sess, cell, initialItems = [20, 15, 76], initialState = cell.zero_state(1, tf.float32)):
    numInitialItems = len(initialItems)
    upTo = np.random.randint(numInitialItems, 80) + 1
    initialItems = tf.constant(np.array([(encodeItem(i)) for i in initialItems]))
    initialState = tf.nn.rnn_cell.LSTMStateTuple(initialState[0], initialState[1])
    localstate = initialState
    for i in range(initialItems.shape[0]):
        output, localstate = cell(tf.cast(tf.reshape(initialItems[i], [1, NUM_ITEMS]), tf.float32), localstate)
    decodedItems = []
    lastItemOneHot = lstmOutputToOneHotItemTensor(sess, output)
    decodedItems.append(decodeItem(sess, lastItemOneHot, itemDict))
    for i in range(upTo - numInitialItems):
        output, localstate = cell(lastItemOneHot, localstate)
        lastItemOneHot = lstmOutputToOneHotItemTensor(sess, output)
        decodedItems.append(decodeItem(sess, lastItemOneHot, itemDict))
    return decodedItems

In [57]:
trainFlag = False
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter('logs/train',
                                      sess.graph)
    test_writer = tf.summary.FileWriter('logs/test')
    if (trainFlag):
        sess.run(init)
        train(sess)
    else:
        print('Doing test')
        saver.restore(sess, ckpoint_dir)
        accuracies_val = test(sess)
        indicesDescendingOrder = np.argsort(accuracies_val)[::-1]
        print(match_ids_test.iloc[indicesDescendingOrder]) 
        print(accuracies_val[indicesDescendingOrder])
        np.random.seed(None)
        with tf.variable_scope("generation"):
            print(generate(sess, cell))
            print(generate(sess, cell))
            print(generate(sess, cell, [44]))

Doing test
INFO:tensorflow:Restoring parameters from /home/nabeel/Documents/Projects/Dota-ItemSequence/model-backups/model.ckpt
Loss: 1.3737977743148804
364    3293379090
158    3301460616
24     3308683837
116    3323479928
319    3319731855
228    3296366555
427    3310878290
73     3309528239
340    3324926058
80     3300680970
395    3315473440
194    3321646455
337    3319076943
70     3294725255
300    3293295897
280    3314578325
249    3320590560
414    3314105690
444    3301613386
261    3297219091
279    3290223482
208    3318984373
338    3297130597
40     3307127540
127    3315779611
39     3301737194
211    3325652705
209    3297160891
107    3314443977
193    3321957746
          ...    
405    3291094213
366    3291011617
274    3319411496
46     3293397819
452    3324854173
400    3309550729
112    3311429433
78     3322012917
453    3306692533
85     3305555302
362    3309861386
104    3289269945
301    3311768905
241    3303632981
68     3300394105
255    3304042828
3